# Solving the Phillips-Wunsch-Garrett IVP with non-constant coefficients using Dedalus

In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py
from dedalus import public as de
from dedalus.extras import flow_tools
from dedalus.extras import plot_tools
from dedalus.tools import post
import pathlib
import time
from IPython import display

import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")

logger = logging.getLogger(__name__)

# Problem formulation

### Physical parameters

In [17]:
N = 1.3e-3
f = 0.53e-4

# topographic parameters
slopeAngle = 2.e-3
tht = slopeAngle

# mixing parameters
d = 230
k0 = 5.2e-5
k1 = 1.8e-3
Pr = 1

In [18]:
def calc_delta():
    return ((4*Pr**2*(k0+k1)**2) / (f**2*np.cos(tht)**2 * (1 + (N**2*np.tan(tht)**2/(f**2))*Pr)))**0.25

In [19]:
#===== Set up domain =====
# Create basis and domain for all experiments
Lz = 4000
nz = 512
z_basis = de.Chebyshev('z', nz, interval=(0, Lz), dealias=3/2)
domain = de.Domain([z_basis],np.float64)
z = domain.grid(0)

#===== Set up Problem ======
problem = de.IVP(domain, variables=['u', 'v', 'b', 'uz', 'vz', 'bz']);

# Set parameters
problem.parameters['f'] = f
problem.parameters['tht'] = tht # set in loop
problem.parameters['N'] = N
k = domain.new_field(name='k')
k['g'] = k0+k1*np.exp(-z/d)
problem.parameters['k'] = k

nu = domain.new_field(name='nu')
nu['g'] = (k0+k1*np.exp(-z/d))*Pr
problem.parameters['nu'] = nu

# Main equations
problem.add_equation("-f*cos(tht)*v - sin(tht)*b - dz(nu*uz) = 0")
problem.add_equation("f*u*cos(tht) - dz(nu*vz) = 0")
problem.add_equation("dt(b) + N**2*sin(tht)*u - dz(k*bz) = N**2*cos(tht)*dz(k)")

# Auxiliary equations defining the first-order reduction
problem.add_equation("uz - dz(u) = 0")
problem.add_equation("vz - dz(v) = 0")
problem.add_equation("bz - dz(b) = 0")

# Boundary conditions
problem.add_bc('left(u) = 0')
problem.add_bc('left(v) = 0')
problem.add_bc('left(bz) = - N**2*cos(tht)')
problem.add_bc('right(uz) = 0')
problem.add_bc('right(vz) = 0')
problem.add_bc('right(bz) = 0')

# Set solver for IVP
solver = problem.build_solver(de.timesteppers.RK443);

#==== Set initial conditions ====
# Reference local grid and state fields
z = domain.grid(0)
u = solver.state['u']
v = solver.state['v']
b = solver.state['b']
uz = solver.state['uz']
vz = solver.state['vz']
bz = solver.state['bz']

# State from a state of rest
u['g'] = np.zeros_like(z)
v['g'] = np.zeros_like(z)
b['g'] = np.zeros_like(z)
u.differentiate('z', out=uz)
v.differentiate('z', out=vz)
b.differentiate('z', out=bz)


#==== Create analysis files ====
output_path = '../../data/dedalus/basic_state'
analysis = solver.evaluator.add_file_handler(output_path, iter=10*365)
analysis.add_system(solver.state, layout='g')

# Stop stopping criteria
solver.stop_sim_time = (5000.*365.*24.*60.*60.)
solver.stop_wall_time = np.inf
solver.stop_iteration = np.inf

# Main loop
dt = 365*24*60.*60.
start_time = time.time()
print('Years: ')
while solver.ok:
    solver.step(dt);
    if solver.sim_time % (100*365.*24.*60.*60.) == 0:
        print(solver.sim_time/(365.*24.*60.*60.), end=", ")
        
end_time = time.time()
print('Runtime:', end_time-start_time)
z_da = domain.grid(0, scales=domain.dealias)

2020-01-27 10:30:53,139 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 9.4e+00/s
Years: 


/home/hdrake/.conda/envs/bblturb/lib/python3.7/site-packages/dedalus/core/evaluator.py:533: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  dset.dims.create_scale(scale, sn)
/home/hdrake/.conda/envs/bblturb/lib/python3.7/site-packages/dedalus/core/evaluator.py:550: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  dset.dims.create_scale(scale, lookup)


100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1600.0, 1700.0, 1800.0, 1900.0, 2000.0, 2100.0, 2200.0, 2300.0, 2400.0, 2500.0, 2600.0, 2700.0, 2800.0, 2900.0, 3000.0, 3100.0, 3200.0, 3300.0, 3400.0, 3500.0, 3600.0, 3700.0, 3800.0, 3900.0, 4000.0, 4100.0, 4200.0, 4300.0, 4400.0, 4500.0, 4600.0, 4700.0, 4800.0, 4900.0, 5000.0, 2020-01-27 10:32:11,281 solvers 0/1 INFO :: Simulation stop time reached.
Runtime: 78.13199400901794


In [20]:
from dedalus.tools import post
import pathlib
import os
post.merge_process_files(output_path, cleanup=True)
set_paths = list(pathlib.Path(output_path).glob("basic_state_s*.h5"))
import os
os.system(f"rm -f {output_path}/output.h5")
post.merge_sets(output_path+"/output.h5", set_paths, cleanup=True)

2020-01-27 10:32:11,290 post 0/1 INFO :: Merging files from ../../data/dedalus/basic_state
2020-01-27 10:32:11,442 post 0/1 INFO :: Creating joint file ../../data/dedalus/basic_state/output.h5


/home/hdrake/.conda/envs/bblturb/lib/python3.7/site-packages/dedalus/tools/post.py:230: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  joint_dset.dims.create_scale(scale, scalename)
/home/hdrake/.conda/envs/bblturb/lib/python3.7/site-packages/dedalus/tools/post.py:349: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  joint_dset.dims.create_scale(scale, scale_name)
